In [ ]:
using PowerModelsDistribution
using XLSX
using DataFrames
using Plots
using NBInclude
@nbinclude("reduce_size.ipynb")
@nbinclude("plot_network.ipynb")

struct power_network
    name
    load_data_dir
    pv_data_dir
    network_data_dir
    orig_eng
    eng
    load_profiles
    pv_profile

    function power_network(name::String)
        load_data_dir = joinpath(name, "Load_Profiles.xlsx")
        sheet_name = "Sheet1" 
        load_profiles = XLSX.readtable(load_data_dir, sheet_name) |> DataFrame

        pv_data_dir = joinpath(name, "PV_Profile.xlsx")
        sheet_name = "Sheet1" 
        pv_profile = XLSX.readtable(pv_data_dir, sheet_name) |> DataFrame

        network_data_dir = joinpath(name, "feeder.dss")
        eng = parse_file(network_data_dir)
        reduced_eng = deepcopy(eng)
        reduce_size(reduced_eng)

        return new(name, load_data_dir, pv_data_dir, network_data_dir, eng, reduced_eng, load_profiles, pv_profile)
    end
end


function plot_load_profiles!(pn::power_network; subplot::Union{Int, Nothing} = nothing)
    load_profiles = pn.load_profiles
    loads = 1:size(load_profiles, 2)
    
    for load in loads
        if subplot !== nothing
            # Plot in the specified subplot
            plot!(subplot = subplot, load_profiles[:, load], label = "")
        else
            # Create a new standalone plot
            plot!(load_profiles[:, load], label = "")
        end
    end

    if subplot !== nothing
        xlabel!(subplot = subplot, "Time")
        ylabel!(subplot = subplot, "Power (kW)")
        title!(subplot = subplot, "Load profile of customers")
    else
        xlabel!("Time")
        ylabel!("Power (kW)")
        title!("Load profile of customers")
    end
end


function plot_reduced_and_original(pn::power_network)
    eng = pn.orig_eng
    reduced_eng = pn.eng

    p = Plots.plot([0.0, 10.0, 9.0, 9.0, 10.0],[0.0, 0.0, -0.7, 0.7, 0.0], linewidth=10, lc=:black, grid=false, showaxis=false, label=false, ylims=(-10,10))
    annotate!(p, 5, 3, ("Reduction","Times"), valign=:center, halign=:center)

    layout = @layout([a{0.4w} b{0.2w} c{0.4w}])
    p_comb = Plots.plot(plot_network(eng), p, plot_network(reduced_eng), layout=layout, size=(800, 300))
    display(p_comb)
end

plot_network (generic function with 1 method)